---
layout: post
title: Sprint 5 beta Blog
description:  Leaderboard
type: issues 
comments: true
---

### Group's Purpose

The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.

### My Purpose

My purpose is to design a leaderboard so that people can compete with their friends in order to achieve a higher score than them in certain drawings. This allows people to interact with one another more and have a little brainbreak to relieve them of their stress a little


### My contribution
Designed a leaderboard with CRUD operation support that can also be tested in postman and has a database to be saved and backed up

### Design
User submit their username, their drawing they did, and the score they got for the drawing (this depends on the amount of times they guessed it correct). They have the free will to change it however they want. even cheat a little :) 

### Create
To add a new leaderboard entry, use the POST method at the /api/leaderboard endpoint. This endpoint expects a JSON payload with profile_name, drawing_name, and score.

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['POST'])
def add_leaderboard_entry():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No data provided"}), 400

        entry = LeaderboardEntry(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=int(data.get('score', 0))
        )
        
        if entry.create():
            return jsonify({"message": "Entry added successfully"}), 201
        return jsonify({"error": "Failed to add entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500

### Read
To retrieve all leaderboard entries, use the GET method at the /api/leaderboard endpoint. This endpoint returns a JSON array of all entries sorted by score in descending order.

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    try:
        entries = LeaderboardEntry.query.order_by(
            LeaderboardEntry.score.desc()
        ).all()
        return jsonify([entry.read() for entry in entries]), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

### Update
To update an existing leaderboard entry, use the PUT method at the /api/leaderboard endpoint. This endpoint expects a JSON payload with profile_name, drawing_name, and score.

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['PUT'])
def update_leaderboard_entry():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No data provided"}), 400

        existing_entry = LeaderboardEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name')
        ).first()

        new_score = int(data.get('score', 0))

        if existing_entry:
            existing_entry.score = new_score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        entry = LeaderboardEntry(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=new_score
        )
        
        if entry.create():
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500

### Delete Operation
To delete a specific leaderboard entry, use the DELETE method at the /api/leaderboard/<profile_name>/<drawing_name> endpoint. This endpoint deletes the entry matching the provided profile_name and drawing_name.


In [ ]:
@leaderboard_api.route('/api/leaderboard/<profile_name>/<drawing_name>', methods=['DELETE'])
def delete_leaderboard_entry(profile_name, drawing_name):
    try:
        entry = LeaderboardEntry.query.filter_by(
            profile_name=profile_name,
            drawing_name=drawing_name
        ).first()
        
        if not entry:
            return jsonify({"error": "Entry not found"}), 404
            
        entry.delete()
        return jsonify({"message": "Entry deleted successfully"}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500

### Call to Algorithm Request
In the frontend, we use the fetch API to make requests to the backend. For example, to submit a score, we send a PUT request with the profile name, drawing name, and score. The response is handled by checking the status and updating the DOM accordingly. If the request is successful, we update the leaderboard; if there’s an error, we display an error message.

In [ ]:
async function submitScore() {
    const profileName = document.getElementById('profileName').value.trim();
    const drawingName = document.getElementById('drawingName').value.trim();
    const score = parseInt(document.getElementById('score').value);

    if (!profileName || !drawingName) {
        showMessage('Please fill in all fields', true);
        return;
    }

    if (isNaN(score) || score < 0 || score > 100) {
        showMessage('Please enter a valid score between 0 and 100', true);
        return;
    }

    try {
        const response = await fetch(API_URL, {
            method: 'PUT',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({
                profile_name: profileName,
                drawing_name: drawingName,
                score: score
            })
        });

        const data = await response.json();

        if (response.ok) {
            showMessage(data.message);
            document.getElementById('profileName').value = '';
            document.getElementById('drawingName').value = '';
            document.getElementById('score').value = '';
            await fetchLeaderboard();
        } else {
            throw new Error(data.error || 'Failed to submit score');
        }
    } catch (error) {
        console.error('Error:', error);
        showMessage(error.message, true);
    }
}